# Welcome to TQ42

## Example of Multilayer Perceptron (MLP)

In [12]:
from tq42.client import TQ42Client
from tq42.organization import list_all as list_all_organizations
from tq42.project import list_all as list_all_projects
from tq42.experiment import list_all as list_all_experiments
from tq42.experiment_run import ExperimentRun
from tq42.compute import HardwareProto
from tq42.algorithm import (
    AlgorithmProto, 
    OptimProto, 
    LossFuncProto, 
    DatasetStorageInfoProto, 
    TSTrainInputsProto,
    TSMLPTrainMetadataProto, 
    TSMLPTrainParametersProto,
    ActFuncProto
)
from google.protobuf.json_format import MessageToDict

# Create a client and selecting org, proj and exp

In [13]:

with TQ42Client() as client:
    org_sample = list_all_organizations(client=client)[0] 
    proj_sample = list_all_projects(client=client, organization_id=org_sample.id)[0]
    exp_sample = list_all_experiments(client=client, project_id=proj_sample.id)[-1]

    print(proj_sample)
    print(org_sample)
    print(exp_sample)



# Set parameters for the train

In [14]:

parameters_train = MessageToDict(TSMLPTrainMetadataProto(
    parameters=TSMLPTrainParametersProto(
        input_width=1,
        label_width=1,
        dim_list=[30, 45, 60],
        act_func=ActFuncProto.SIGMOID,
        dropout=True,
        dropout_p=0.5,
        bn=True,
        num_epochs=5,
        batch_size=20,
        learning_rate=0.05,
        optim=OptimProto.ADAM,
        loss_func=LossFuncProto.MSE,
    ),
    inputs=TSTrainInputsProto(
        data=DatasetStorageInfoProto(storage_id="1fcc5e6c-b962-410b-a257-739f7172db03")
    )
), preserving_proto_field_name=True)

# Train the model

In [15]:

run = ExperimentRun.create(
    client=client,
    algorithm=AlgorithmProto.TS_MLP_TRAIN, 
    experiment_id=exp_sample.id,
    compute=HardwareProto.SMALL, 
    parameters=parameters_train
)

print(run.data)

id: "52516153-2f8e-4da3-8cc1-04e075a6aeb1"
experiment_id: "811fd59b-2b51-42ea-b573-b5aebb0d5d7a"
sequential_id: 17
status: QUEUED
algorithm: TS_MLP_TRAIN
hardware: SMALL
metadata: "{\"parameters\":{\"input_width\":1,\"label_width\":1,\"dim_list\":[30,45,60],\"act_func\":\"SIGMOID\",\"dropout\":true,\"dropout_p\":0.5,\"bn\":true,\"num_epochs\":5,\"batch_size\":20,\"learning_rate\":0.05,\"optim\":\"ADAM\",\"loss_func\":\"MSE\",\"train_model_info\":null},\"inputs\":{\"data\":{\"storage_id\":\"1fcc5e6c-b962-410b-a257-739f7172db03\"}}}"
result {
}
created_by: "35dc8cba-b06d-4f15-b9bc-588f608789de"
created_at {
  seconds: 1705663712
  nanos: 542225914
}


# Import functions for the evaluate

In [16]:
from tq42.algorithm import (
    ModelStorageInfoProto,
    TSEvalInputsProto,
    TSMLPEvalMetadataProto,
    TSMLPEvalParametersProto
) 

# Set parameters for the evaluate

In [17]:
params_eval =  MessageToDict(TSMLPEvalMetadataProto(
    parameters=TSMLPEvalParametersProto(
        input_width=1,
        label_width=1,
        dim_list=[30, 45, 60],
        act_func=ActFuncProto.SIGMOID,
        dropout=True,
        dropout_p=0.5,
        bn=True
    ),
    inputs=TSEvalInputsProto(
        model=ModelStorageInfoProto(storage_id="MODEL_STORAGE_BUCKET_ID"),
        data=DatasetStorageInfoProto(storage_id="DATA_STORAGE_BUCKET_ID")
    )
), preserving_proto_field_name=True)

# Evaluate the model

In [18]:
    run = ExperimentRun.create(
        client=client,
        algorithm=AlgorithmProto.TS_MLP_EVAL,
        experiment_id=exp_sample.id,
        compute=HardwareProto.SMALL,
        parameters=params_eval
    )
print(run.data)

id: "501e8bb5-428f-4461-9bf3-6f836af57916"
experiment_id: "811fd59b-2b51-42ea-b573-b5aebb0d5d7a"
sequential_id: 18
status: QUEUED
algorithm: TS_MLP_EVAL
hardware: SMALL
metadata: "{\"parameters\":{\"input_width\":1,\"label_width\":1,\"dim_list\":[30,45,60],\"act_func\":\"SIGMOID\",\"dropout\":true,\"dropout_p\":0.5,\"bn\":true},\"inputs\":{\"model\":{\"storage_id\":\"MODEL_STORAGE_BUCKET_ID\"},\"data\":null}}"
result {
}
created_by: "35dc8cba-b06d-4f15-b9bc-588f608789de"
created_at {
  seconds: 1705663717
  nanos: 709899103
}
